Importing Libraries and Using Kaggle API to access dataset

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 15.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import datasets
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel
import pandas as pd
import tensorflow
import transformers
import tensorflow as tf
from tensorflow.keras import layers
import re

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d samanazhar/multilingual-sentiment-analysis

100% 1.81G/1.81G [00:20<00:00, 88.6MB/s]
100% 1.81G/1.81G [00:20<00:00, 94.7MB/s]


In [ ]:
!unzip /content/multilingual-sentiment-analysis.zip

Archive:  /content/multilingual-sentiment-analysis.zip
  inflating: PMLN_predicted_tweets.csv  
  inflating: PPP_predicted_tweets.csv  
  inflating: PTI_predicted_tweets.csv  
  inflating: Scraped_Tweets/PMLN_Complete_Dataset.csv  
  inflating: Scraped_Tweets/PPP_Complete_Dataset.csv  
  inflating: Scraped_Tweets/PTI_Complete_Dataset.csv  


In [ ]:
!pip install transformers

In [ ]:
pip install transformers tensorflow


In [ ]:
import datasets
import pandas as pd
import tensorflow as tf
import transformers
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification

Importing Datasets

In [ ]:
ds=pd.read_csv('/content/PMLN_predicted_tweets.csv')
ds1=pd.read_csv('/content/PPP_predicted_tweets.csv')
ds2=pd.read_csv('/content/PTI_predicted_tweets.csv')

In [ ]:
ds = ds[ds['language'] == 'in'].dropna()
ds1 = ds1[ds1['language'] == 'in'].dropna()
ds2 = ds2[ds2['language'] == 'in'].dropna()

# Concatenate the data frames
df = pd.concat([ds, ds1, ds2], ignore_index=True)


Preprocessing Data using mBERT tokens

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
max_len = 64  # Adjust the maximum sequence length as needed

# Tokenize the text and convert it to a format suitable for BERT
encoded_texts = tokenizer(df['preprocessed_tweet'].tolist(), truncation=True, padding='max_length', max_length=max_len, return_tensors='tf')


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(encoded_texts['input_ids'].numpy(), df['sentiment'], test_size=0.1, random_state=10, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=21, shuffle=True)


mBERT Model

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased")
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  177853440 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 177854978 (678.46 MB)
Trainable params: 177854978 (678.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
es_callback = EarlyStopping(monitor='val_accuracy', patience=3)
model.fit(
    x_train, y_train,
    epochs=2,
    batch_size=50,
    validation_data=(x_val, y_val),
    callbacks=[es_callback]
)


Epoch 1/2
2043/2043 [==============================] - 1262s 594ms/step - loss: 7.5237 - accuracy: 0.4883 - val_loss: 7.5834 - val_accuracy: 0.4916
Epoch 2/2
2043/2043 [==============================] - 1244s 609ms/step - loss: 7.5227 - accuracy: 0.4877 - val_loss: 7.5834 - val_accuracy: 0.4916


In [ ]:
# Save the model to a directory
model.save_pretrained("path_to_save_directory")

# Optionally, save the tokenizer as well
tokenizer.save_pretrained("path_to_save_directory")


('path_to_save_directory/tokenizer_config.json',
 'path_to_save_directory/special_tokens_map.json',
 'path_to_save_directory/vocab.txt',
 'path_to_save_directory/added_tokens.json')

In [ ]:
# Save the model's weights
model.save_weights("model_weights.h5")

# Save the entire model (architecture and weights) in TensorFlow SavedModel format
model.save("path_to_saved_model_directory")


In [ ]:
from transformers import BertTokenizer
import numpy as np

# Define the input sentence
sentence = ["usse maaro!!"]

# Tokenize the input sentence
tokenized_sentence = tokenizer(sentence, truncation=True, padding='max_length', max_length=max_len, return_tensors='tf')

# Make predictions
output = model.predict(tokenized_sentence['input_ids'].numpy())

# Get the model's output logits for binary classification
logits = output.logits

# Define the threshold for binary classification (adjust as needed)
threshold = 0.3

# Determine the sentiment prediction
predictions = (logits > threshold).astype(int)

# Print the sentiment prediction
print("Sentiment for this tweet is:", predictions[0][0])


1/1 [==============================] - 0s 67ms/step
Sentiment for this tweet is: 1


In [ ]:
from transformers import BertTokenizer
import numpy as np

# Define the input sentence
sentence = ["i will vote for you!!"]

# Tokenize the input sentence
tokenized_sentence = tokenizer(sentence, truncation=True, padding='max_length', max_length=max_len, return_tensors='tf')

# Make predictions
output = model.predict(tokenized_sentence['input_ids'].numpy())

# Get the model's output logits for binary classification
logits = output.logits

# Define the threshold for binary classification (adjust as needed)
threshold = 0.3

# Determine the sentiment prediction
predictions = (logits > threshold).astype(int)

# Print the sentiment prediction
print("Sentiment for this tweet is:", predictions[0][0])


1/1 [==============================] - 0s 52ms/step
Sentiment for this tweet is: 1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
model.save_weights('/content/drive/My Drive/my_model_weights.h5')

In [ ]:
import os
print(os.listdir('/content/drive/My Drive/'))


['Tarunpanda1.jpg', 'TarunPanda.jpg', '2019_01_20 7_06 PM Office Lens', 'cisce (1).org-SSCER-186620897.pdf', 'in.gov.cbse-HSCER-156057062020.pdf', 'Document1.gdoc', 'oci', 'hall ticket', 'aadhar', 'cbse hall ticket.pdf', 'aadhar card.pdf', 'cisce.org-SSCER-186620897.pdf', 'in.gov.cbse-HSCER-156057062020 (1).pdf', 'passport.pdf', 'oci.pdf', 'Untitled form (1).gform', 'Untitled form.gform', 'aadhar (1).pdf', 'aadhar.pdf', 'sharmila_aadhar.pdf', "List of shows I've Watched:.gdoc", 'research papa for myvee.gdoc', 'tc20bce1772.pdf', 'marksheet20bce1772.pdf', '5-11-22, 11:20 AM Microsoft Lens.pdf', 'aadhar_back.pdf', 'List of Shows To Watch.gdoc', 'Colab Notebooks', 'research club names.gdoc', 'Research club.docx', 'EDA Review 1:.gdoc', 'Financial Modelling Project - Janhvi.gdoc', 'WasteClassification_TARP_Group3.gdoc', 'room_visitors.xlsx', 'OM LOR.gdoc', 'OM LOR.docx', 'FRSA LOR.gdoc', 'FRSA LOR.docx', 'Copy of FRSA LOR.docx', 'AY-2020-2021_6.12.2021.pdf', 'SIN J Comp Review 1.gdoc', 'port

In [ ]:
model.load_weights('/content/drive/My Drive/my_model_weights.h5')
